In [1]:
import numpy as np
from math import *
from random import shuffle

In [7]:
RDD_user_labels = sc.textFile("contest_dataset_label/*") \
    .map(lambdxxa x: x.split()) \
    .map(lambda x: (x[0], (int(x[1]) - 1, int(x[2]) - 1)))
RDD_user_labels.count()

11439408

In [29]:
all_gender = RDD_user_labels \
    .map(lambda x: (x[1][0], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: x[1]) \
    .collect()
all_gender = np.array(all_gender)
all_gender

array([7634428, 3804980])

In [30]:
all_age = RDD_user_labels \
    .map(lambda x: (x[1][1], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: x[1]) \
    .collect()
all_age = np.array(all_age)
all_age

array([2079603, 1736735, 1473666, 2682296, 2214497,  696202,  556409])

In [33]:
all_gender = 1. * all_gender / sum(all_gender)
all_age = 1. * all_age / sum(all_age)

np.save("all_dist", {'gender': all_gender, 'age': all_age})

In [2]:
x = np.load('all_dist.npy').item()
all_gender, all_age = x['gender'], x['age']


In [10]:
RDD_url = sc.textFile("contest_dataset_url/*") \
    .map(lambda x: x.split()) \
    .map(lambda x: (x[0], int(x[1]) - 1))m


In [3]:
x = np.load('tmp_url/freq.npy').item()
freq_gender, freq_age = x['gender'], x['age']
T = np.sum(freq_gender, axis=1)
possible_urls = np.where(T > 10000)[0]

In [4]:
'''
determine `good` app_id.

criterion: 
  1. number of users must be larger than MIN_USERS
  2. (DEPRECATED) corresponding distribution is `significantly` far from overall_distribution
     `significantly`: distance > AGE_THRES (for age) or distance > GENDER_THRES (for gender)
     distance measurement: KL divergence
  3. picking the apps with largest distance from overall distribution.
'''

MIN_USERS = 10000
MAX_URLS = 2000000
REG_DIM = 300 # dimension of logistic regression

gender_dists = []
age_dists = []
for url_id in possible_urls:
    gender, age = freq_gender[url_id], freq_age[url_id]
    if sum(gender) < MIN_USERS:
        continue
    population = sum(gender)
    gender = 1. * gender / population
    age = 1. * age / population
    
    extra = log(population) * 0.001
    gender_dists += [(np.sum(gender * np.log(gender / all_gender + 1e-20)) + extra, url_id)]
    age_dists += [(np.sum(age * np.log(age / all_age + 1e-20)) + extra, url_id)]

gender_dists = list(reversed(sorted(gender_dists)))[:REG_DIM]
age_dists = list(reversed(sorted(age_dists)))[:REG_DIM]

good_gender_url_map = np.zeros(MAX_URLS) - 1
good_age_url_map = np.zeros(MAX_URLS) - 1
for i in range(REG_DIM):
    good_gender_url_map[gender_dists[i][1]] = i
    good_age_url_map[age_dists[i][1]] = i



In [5]:
testset = sc.textFile("contest_testset/*").collect()

In [8]:
'''
partition all possible user_id into 3 sets
1. testset
2. trainset
3. devset

user_id's in devset are used for cross validtion

criterion to partition: random shuffle all user_id, devset owns the first `devset_size` user_ids.

'''

in_testset = {}
in_trainset = {}
in_devset = {}
trainset_lists = []

devset_size = 1000000

for x in testset:
    in_testset[x] = True

all_labels = RDD_user_labels.map(lambda x: x[0]).collect()
shuffle(all_labels)

for i, user_id in enumerate(all_labels):
    if i < devset_size:
        in_devset[user_id] = True
    in_trainset[user_id] = True
trainset_lists = in_trainset.keys()


In [11]:
tmp = {}
for i in in_testset.keys() + in_devset.keys():
    tmp[i] = True

RDD_results = RDD_url.filter(lambda x: x[0] in tmp and (good_gender_url_map[x[1]] != -1 or good_age_url_map[x[1]] != -1)).persist()

In [ ]:
all_results = RDD_results.collect()

In [85]:
sc.set

[(u'hive.metastore.warehouse.dir',
  u'file:/home/xiaomi/\u684c\u9762/spark-warehouse'),
 (u'spark.sql.catalogImplementation', u'hive'),
 (u'spark.app.id', u'local-1474281421054'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.driver.maxResultSize', u'100g'),
 (u'spark.driver.host', u'10.237.9.72'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]'),
 (u'spark.executor.id', u'driver'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.driver.memory', u'100g'),
 (u'spark.app.name', u'PySparkShell'),
 (u'spark.driver.port', u'41997')]